
# PTZ Hedeflerinden Snapshot Alma (ISAPI / Digest Auth)

Bu notebook, **Hikvision ISAPI** uyumlu bir IP kameraya **HTTP Digest Auth** ile bağlanır,
tanımladığınız **PTZ hedef konumlarına** kamerayı taşır ve her konumda **snapshot** (tek kare fotoğraf) alır.


## 1) Kurulum

In [ ]:
!pip install requests

## 2) Ayarlar

In [ ]:

# Bu hücrede kamera bağlantısı, kanal kimlikleri ve bazı davranış ayarlarını yapıyoruz.
# Değerleri kendi cihazınıza göre güncelleyin.

import os
from datetime import datetime


CAMERA_HOST         = "10.85.4.100:80"  # IP
USERNAME            = "datamind"    # <- kullanıcı adı
PASSWORD            = "Data321321."    # <- şifre

# PTZ ve yayın kanalları genelde birbirinden farklıdır:
# - PTZ için çoğu cihazda kanal 1 kullanılır
# - Snapshot almak için Streaming kanalları 101 (main) / 102 (sub) vb. olabilir
PTZ_CHANNEL_ID      = 1
STREAM_CHANNEL_ID   = 101

# Snapshot'ların kaydedileceği klasör
SAVE_DIR            = "snapshots"

# Kamera hedefe giderken biraz süre tanımak gerekir; sabit bir bekleme koyuyoruz
MOVE_SETTLE_SECONDS = 5

# Bazı cihazlar absolute PTZ değerlerini 0.1° cinsinden bekler; o zaman ×10 göndermek gerekir.
AZIMUTH_SCALE       = 10
ELEVATION_SCALE     = 10
ZOOM_SCALE          = 10

# Örnek hedefler: isim -> (azimuth, elevation, zoom)
PTZ_TARGETS = {
    "karpuz": {"azimuth": 319, "elevation": 21, "zoom": 2},
    "portakal":    {"azimuth":  10, "elevation": 26, "zoom": 2},
    "mandalina1":    {"azimuth":  343, "elevation": 20, "zoom": 4},
    "mandalina2":    {"azimuth":  342, "elevation": 29, "zoom": 4},
    "muz":    {"azimuth":  41, "elevation": 19, "zoom": 2},
    "salkım":    {"azimuth":  0, "elevation": 0, "zoom": 8},
    "biber":  {"azimuth":   10, "elevation":  0, "zoom": 9},
    "patates":  {"azimuth":   22, "elevation":  0, "zoom": 10},
    "lahana":  {"azimuth":   358, "elevation":  -4, "zoom": 16},
    
}

# Yardımcı: çıktı klasörünü oluştur
os.makedirs(SAVE_DIR, exist_ok=True)

print("Ayarlar yüklendi. PTZ kanal:", PTZ_CHANNEL_ID, "| Streaming kanal:", STREAM_CHANNEL_ID)


Ayarlar yüklendi. PTZ kanal: 1 | Streaming kanal: 101


## 3) Bağlantı ve Kimlik Doğrulama

In [ ]:

# Burada requests ve DigestAuth oturumunu hazırlıyoruz.
# Aynı oturumu tekrar tekrar kullanmak ufak da olsa performans ve basitlik sağlar.

import requests
from requests.auth import HTTPDigestAuth

BASE_URL = f"http://{CAMERA_HOST}/ISAPI"
AUTH     = HTTPDigestAuth(USERNAME, PASSWORD)
SESSION  = requests.Session()

# Küçük bir yardımcı: HTTP sonucunun "başarılı" sayılıp sayılmadığını kontrol etmek
def _is_ok(resp: requests.Response) -> bool:
    return 200 <= resp.status_code < 300

print("Hazır. BASE_URL:", BASE_URL)


Hazır. BASE_URL: http://10.85.4.100:80/ISAPI


## 4) PTZ'yi Mutlak Konuma Taşıma

In [ ]:

# Bu fonksiyon kamerayı mutlak (absolute) azimuth/elevation/zoom değerlerine götürür.

def ptz_move_absolute(azimuth: float, elevation: float, zoom: float, timeout_s: float = 5.0) -> None:
    url = f"{BASE_URL}/PTZCtrl/channels/{PTZ_CHANNEL_ID}/Absolute"
    xml_body = f"""<?xml version="1.0" encoding="UTF-8"?>
<PTZData>
  <AbsoluteHigh>
    <azimuth>{azimuth * AZIMUTH_SCALE}</azimuth>
    <elevation>{elevation * ELEVATION_SCALE}</elevation>
    <absoluteZoom>{zoom * ZOOM_SCALE}</absoluteZoom>
  </AbsoluteHigh>
</PTZData>"""

    try:
        resp = SESSION.put(
            url, data=xml_body, auth=AUTH,
            headers={"Content-Type": "application/xml"},
            timeout=timeout_s
        )
        if not _is_ok(resp):
            raise RuntimeError(f"PTZ Absolute hata: {resp.status_code} - {resp.text[:200]}")
    except requests.RequestException as e:
        raise RuntimeError(f"PTZ isteği başarısız: {e}") from e

print("PTZ absolute fonksiyonu yüklendi.")


PTZ absolute fonksiyonu yüklendi.


## 5) Snapshot Alma

In [ ]:
# === SAATLİK KLASÖRLEME İÇİN GÜNCELLENMİŞ SNAPSHOT FONKSİYONU ===
import os
from datetime import datetime

def take_snapshot(name_prefix: str) -> str:
    """
    Streaming kanalından tek kare fotoğraf çeker ve şu yapıda kaydeder:
    snapshots/YYYY-MM-DD/HH/<isim>_HHMMSS.jpg
    """
    url = f"{BASE_URL}/Streaming/channels/{STREAM_CHANNEL_ID}/picture"
    try:
        resp = SESSION.get(url, auth=AUTH, stream=True, timeout=10)
        if not _is_ok(resp):
            raise RuntimeError(f"Snapshot hata: {resp.status_code} - {resp.text[:200]}")

        now     = datetime.now()
        ts_date = now.strftime("%Y-%m-%d")   # Gün klasörü
        ts_hour = now.strftime("%H")         # Saat klasörü
        ts_time = now.strftime("%H%M%S")     # Dosya adı saat damgası

        day_dir  = os.path.join(SAVE_DIR, ts_date)      # snapshots/2025-10-16
        hour_dir = os.path.join(day_dir, ts_hour)       # snapshots/2025-10-16/21
        os.makedirs(hour_dir, exist_ok=True)

        fname = f"{name_prefix}_{ts_time}.jpg"          # karpuz_211530.jpg
        fpath = os.path.join(hour_dir, fname)

        with open(fpath, "wb") as f:
            for chunk in resp.iter_content(8192):
                if chunk:
                    f.write(chunk)

        # Basit bütünlük kontrolü
        if os.path.getsize(fpath) < 5_000:  # 5 KB altı boş/bozuk olabilir
            print("[uyarı] Snapshot dosyası çok küçük görünüyor:", fpath)

        print(f"[✓] Kaydedildi: {fpath}")
        return fpath

    except requests.RequestException as e:
        raise RuntimeError(f"Snapshot isteği başarısız: {e}") from e


## 6) Hedeflere Git ve Fotoğrafları Al

In [ ]:

# Aşağıdaki hücre her hedef için sırasıyla:
# 1) PTZ'yi absolute konuma götürür
# 2) Kameranın hareketinin bitmesi için kısa bir bekleme yapar
# 3) Snapshot alır ve diske kaydeder

import time

print("[i] Hedefler:", ", ".join(PTZ_TARGETS.keys()))
for name, coords in PTZ_TARGETS.items():
    az = coords["azimuth"]
    el = coords["elevation"]
    zz = coords["zoom"]

    print(f"\n[→] {name}: az={az}, el={el}, zoom={zz} konumuna gidiliyor...")
    ptz_move_absolute(azimuth=az, elevation=el, zoom=zz)

    print(f"[i] PTZ yerleşmesi için {MOVE_SETTLE_SECONDS}s bekleniyor...")
    time.sleep(MOVE_SETTLE_SECONDS)

    print("[→] Snapshot alınıyor...")
    take_snapshot(name_prefix=name)

print("\n[✓] Tüm hedefler tamamlandı.")
